In [20]:
import torch
# import torchvision.transforms as transforms
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torchvision.transforms as T

In [21]:
device = "cuda"

In [22]:
trans = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(0.5, 0.5)
])

In [23]:
train_set = torchvision.datasets.ImageFolder('/kaggle/input/cat-and-dog/training_set/training_set', transform=trans)
test_set  = torchvision.datasets.ImageFolder('/kaggle/input/cat-and-dog/test_set/test_set', transform=trans)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=12, shuffle=True, num_workers=4, pin_memory=True)
test_loader  = torch.utils.data.DataLoader(test_set, batch_size=12, num_workers=4, pin_memory=True)


In [24]:
def compute_accuracy(model, test_loader, threshold=0.5, device='cuda'):
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        losses = []
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images).squeeze(1)
            predictions = (outputs > threshold).float()
            labels = labels.float()
            loss = criterion(outputs, labels)
            total += labels.size(0)
            correct += (predictions == labels).sum().item()
            losses.append(loss.item())
    accuracy = correct / total
    total_loss = sum(losses)/len(losses)
    return accuracy, total_loss


In [25]:
import torch.nn as nn# layers
import torchvision.transforms  as T# resize



class _conv_block(nn.Module):
    def __init__(self, in_channels, out_channels, a):
        super().__init__()

        out_channels = int(out_channels*a)

        self.ConvBlock = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 2, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )


    def forward(self, x):
        return self.ConvBlock(x)




class _depth_wise_separable_convolution(nn.Module):
    def __init__(self, in_channels, out_channels, stride, a):
        super().__init__()

        in_channels = int(in_channels*a)
        out_channels = int(out_channels*a)

        self.dwsc_layer = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, 3, stride, 1, groups=in_channels),
            nn.BatchNorm2d(in_channels),
            nn.ReLU(),
            nn.Conv2d(in_channels, out_channels, 1, 1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )


    def forward(self, x):
        return self.dwsc_layer(x)






class MobileNet(nn.Module):
    def __init__(self, in_channels, a, p):
        super().__init__()

        self.a = a
        self.p = p
        self.n_dwsc_layers = 13
        self.input_size = int(self.p*224)
        
        self.base_model_in_features = [32, 64, 128, 128, 256, 256, *[512 for _ in range(6)], 1024, 1024]
        self.strides = [1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1]

        self.flattened_size = int(1024*self.a)


        self.ConvBlock = _conv_block(in_channels, self.base_model_in_features[0], self.a)

        self.dwsc_layers = nn.ModuleList([
            _depth_wise_separable_convolution(
                self.base_model_in_features[i],
                self.base_model_in_features[i+1],
                self.strides[i],
                self.a
            ) for i in range(self.n_dwsc_layers)
        ])

        self.Pool = nn.AdaptiveAvgPool2d(1) ## 1 x 1 x 1024 -> 1024, 1

        self.Flattener = nn.Flatten()

        self.FC1 = nn.Linear(self.flattened_size, 1)
        
        self.Output = nn.Sigmoid()

        self.Resizer = T.Resize((self.input_size, self.input_size))



    def forward(self, x):
        if self.input_size != 224:
            x = self.Resizer(x)
        
        x = self.ConvBlock(x)

        for i in range(self.n_dwsc_layers):
            x = self.dwsc_layers[i](x)

        x = self.Pool(x)
        x = self.Flattener(x)
        x = self.FC1(x)
        x = self.Output(x)

        return x



In [26]:
model = MobileNet(3, 0.75, 0.714).to(device)

In [27]:
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.BCELoss()
EPOCHS = 40

train_losses, test_losses, train_accuracies, test_accuracies = [], [], [], []

for i in range(EPOCHS):
    losses = []
    model.train()
    for j, item in enumerate(train_loader, 0):
        optimizer.zero_grad()
        images, labels = item[0].to(device), item[1].to(device)
        outputs = model(images)
        outputs = outputs.squeeze(1)
        # print(outputs)
        # print(labels)
        labels = labels.float()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    
    validation_accuracy, validation_loss = compute_accuracy(model, test_loader, 0.5)
    train_accuracy, train_loss = compute_accuracy(model, train_loader, 0.5)
    train_loss = sum(losses)/len(losses)

    train_losses.append(train_loss)
    test_losses.append(validation_loss)
    train_accuracies.append(train_accuracy)
    test_accuracies.append(validation_accuracy)
    
    print(f"train loss: {train_loss}, train accuracy: {train_accuracy}, test loss: {validation_loss}, test accuracy: {validation_accuracy}")


train loss: 0.6538685557995728, train accuracy: 0.7028107432854466, test loss: 0.6019247142520882, test accuracy: 0.6826495304003954
train loss: 0.5950355635520941, train accuracy: 0.7429106808244847, test loss: 0.5698690149911056, test accuracy: 0.708353929807217
train loss: 0.5482691515928614, train accuracy: 0.7820112429731418, test loss: 0.5406832206531389, test accuracy: 0.7276322293623332
train loss: 0.5056681020664955, train accuracy: 0.7890068707058089, test loss: 0.5694596337143486, test accuracy: 0.7286208601087494
train loss: 0.47225781206688483, train accuracy: 0.8479700187382886, test loss: 0.4794605279109887, test accuracy: 0.7790410281759763


KeyboardInterrupt: 

In [ ]:
validation_accuracy, validation_loss = compute_accuracy(model, test_loader, 0.5)


In [ ]:
validation_accuracy